In [1]:
from src.utils.extract_utils import average_vectors, gather_activations_from_dataset

/root/activation-additions-large-models/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch, transformers, accelerate, einops, json


In [3]:
from src.utils.model_utils import load_gpt_model_and_tokeniser

In [4]:
model, tokenizer, MODEL_CONFIG = load_gpt_model_and_tokeniser(model_name="meta-llama/Llama-2-7b-hf")

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 1.18MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 538kB/s]
special_tokens_map.json: 100%|██████████| 414/414 [00:00<00:00, 1.04MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 3.82MB/s]
config.json: 100%|██████████| 609/609 [00:00<00:00, 1.48MB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 33.5MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.98G/9.98G [10:20<00:00, 16.1MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 3.50G/3.50G [03:52<00:00, 15.0MB/s]
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.32s/it]
/root/activation-additions-large-models/myenv/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_t

In [5]:
import src.utils.intervention_utils as iu

# Load Datasets

In [6]:
stories = {}

# Open and read the JSON file
with open('datasets/fantasy.json', 'r') as file:
  # Load the JSON data from the file
  dataset_fantasy = json.load(file)

  stories["fantasy"] = dataset_fantasy

with open('datasets/scifi.json', 'r') as file:
  # Load the JSON data from the file
  dataset_scifi = json.load(file)

  stories["scifi"] = dataset_scifi

In [7]:
from src.utils.dataset_utils import read_all_text_files

training_dataset = read_all_text_files("datasets/opentext_subset")

# Cut texts for first 200 tokens
# Determine the cutoff point using the tokenizer
training_dataset = [tokenizer.decode(tokenizer.encode(text)[:200])[4:] for text in training_dataset][:400]

# Creating a Steering Vector

In [8]:
fantasy_activations = gather_activations_from_dataset(
        stories["fantasy"],
        ["layer_hook_names"], 
        model,
        tokenizer, 
        MODEL_CONFIG, 
        len(stories["fantasy"]), 
        False,
        False,
        False,   
    )

Gathering activations: 100%|██████████| 200/200 [00:47<00:00,  4.21it/s]


In [11]:
!nvidia-smi

Sun Nov 19 19:36:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000                On | 00000000:02:00.0 Off |                  Off |
| 59%   40C    P8               16W / 300W|  26931MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
# Make a tensor of the average activation in each layer
average_fantasy_tensor = average_vectors(fantasy_activations, MODEL_CONFIG)


torch.Size([32030, 4096])


In [11]:
!nvidia-smi

Sun Nov 19 19:42:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000                On | 00000000:02:00.0 Off |                  Off |
| 63%   44C    P8               18W / 300W|  26931MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nv

In [13]:
training_activations = gather_activations_from_dataset(
        training_dataset[:300],
        ["layer_hook_names"], 
        model,
        tokenizer, 
        MODEL_CONFIG, 
        len(training_dataset[:300]), 
        False,
        False,
        False,   
    )

Gathering activations:  39%|███▊      | 116/300 [01:07<01:47,  1.71it/s]

: 

In [13]:
!nvidia-smi

Sun Nov 19 19:38:50 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000                On | 00000000:02:00.0 Off |                  Off |
| 85%   53C    P2               74W / 300W|  32231MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
average_training_tensor = average_vectors(training_activations, MODEL_CONFIG)

: 

In [ ]:
average_training_tensor[30]

In [ ]:
average_fantasy_tensor[30]

In [ ]:
fantasy_steering_vector = average_fantasy_tensor - average_training_tensor

In [ ]:
fantasy_steering_vector[30]

In [ ]:
!nvidia-smi

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# Try Steering!

In [ ]:
MODEL_CONFIG["num_layers"]

In [ ]:
outputs = steering_natural_text(
    "Here is a story:", 
    25,
    fantasy_steering_vector[] * 150, 
    model,
    MODEL_CONFIG, 
    tokenizer, 
    max_new_tokens=100, 
    temperature=1.0, 
    freq_penalty=2.0,
    top_p=0.3,
    n_completions=2,
    n_beams=50
)

In [ ]:
outputs["steered"][1]

In [ ]:
tokenizer.decode(outputs["steered"][0].squeeze()[-max:])